In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
train_data , test_data = data.randomSplit([0.8,0.2])

In [9]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [11]:
model = als.fit(train_data)

In [13]:
predictions = model.transform(test_data)

In [14]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27|  1.5439893|
|     31|   1.0|     4|  2.1431432|
|     31|   1.0|    18|  1.8298163|
|     85|   1.0|    26| -1.3584976|
|     85|   3.0|     1| 0.42336568|
|     85|   5.0|    16| 0.07379147|
|     85|   1.0|    23|  1.5009671|
|     85|   4.0|     7|  2.9481215|
|     85|   3.0|    21|  2.9611678|
|     65|   1.0|    16| -3.1705558|
|     53|   3.0|    13|  1.3882992|
|     53|   1.0|     9|  3.4729524|
|     78|   1.0|    22|  0.6111623|
|     78|   1.0|     1| 0.43431932|
|     78|   1.0|     8|  1.0151638|
|     78|   1.0|    24| 0.53954035|
|     34|   3.0|    25|-0.99165326|
|     81|   1.0|    19|  2.6913815|
|     28|   1.0|     5|  3.5622091|
|     28|   2.0|    15|  1.6383789|
+-------+------+------+-----------+
only showing top 20 rows



In [15]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [16]:
rmse = evaluator.evaluate(predictions)

In [19]:
single_user = test_data.filter(test_data['userId']== 11).select(['movieId','userId'])

In [20]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     16|    11|
|     19|    11|
|     22|    11|
|     27|    11|
|     39|    11|
|     43|    11|
|     51|    11|
|     82|    11|
|     88|    11|
|     99|    11|
+-------+------+



In [21]:
recommendation = model.transform(single_user)

In [23]:
recommendation.orderBy('prediction',ascending = False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     22|    11|   3.6754286|
|     39|    11|   3.4664922|
|     19|    11|   3.2591784|
|     27|    11|    3.069601|
|     82|    11|   2.9436774|
|     88|    11|   1.7962489|
|     43|    11|   1.4855759|
|     99|    11|  0.80719906|
|     51|    11|-0.036966585|
|     16|    11| -0.32417488|
+-------+------+------------+

